# Read file

In [1]:
%%writefile romy.txt
First line is first and second is also first.
Second line is third and third is given below.
Below is the fourth line.
Fourth line is this only.
Only fifth one is correct.
Correct and first.


Overwriting romy.txt


In [2]:
file=open('romy.txt', encoding='utf8')
read=file.read()
file.seek(0)
read
word=len(read.split())
print(word)

line=1
for word in read:
    if word=='\n':
        line+=1
print("Number of lines in file is: ",line)



array=[]
for i in range(line):
    array.append(file.readline())
    
array

36
Number of lines in file is:  6


['First line is first and second is also first.\n',
 'Second line is third and third is given below.\n',
 'Below is the fourth line.\n',
 'Fourth line is this only.\n',
 'Only fifth one is correct.\n',
 'Correct and first.']

# Remove punctuation

In [3]:
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
for ele in read:  
    if ele in punc:  
        read = read.replace(ele, " ")  
        

read
read=read.lower()
read

'first line is first and second is also first \nsecond line is third and third is given below \nbelow is the fourth line \nfourth line is this only \nonly fifth one is correct \ncorrect and first '

# Remove stopwords

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


text_tokens = word_tokenize(read)

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

print(tokens_without_sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rameshwaram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['first', 'line', 'first', 'second', 'first', 'second', 'line', 'third', 'third', 'given', 'fourth', 'line', 'fourth', 'line', 'fifth', 'correct', 'correct', 'first']


In [5]:
len(tokens_without_sw)

18

# Remove digits 

In [6]:
tokens=[]
for i in range(len(tokens_without_sw)):
               if tokens_without_sw[i].isdigit()==False:
                   tokens.append(tokens_without_sw[i])
print(len(tokens))
print(tokens)

                   

18
['first', 'line', 'first', 'second', 'first', 'second', 'line', 'third', 'third', 'given', 'fourth', 'line', 'fourth', 'line', 'fifth', 'correct', 'correct', 'first']


# Stemming

In [7]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
for i in range(len(tokens)):
    tokens[i]=ps.stem(tokens[i])
    
print(tokens)

['first', 'line', 'first', 'second', 'first', 'second', 'line', 'third', 'third', 'given', 'fourth', 'line', 'fourth', 'line', 'fifth', 'correct', 'correct', 'first']


# Create set to avoid multiple entries

In [8]:
newset=set(tokens)
newset

{'correct', 'fifth', 'first', 'fourth', 'given', 'line', 'second', 'third'}

# Create inverted index

In [9]:
dt={}

for i in range(line):
    check=array[i].lower()
    for item in newset:
        
        if item in check:
            if item not in dt:
                dt[item]=[]
                
            if item in dt:
                dt[item].append(i+1)
                
dt



{'first': [1, 6],
 'line': [1, 2, 3, 4],
 'second': [1, 2],
 'third': [2],
 'given': [2],
 'fourth': [3, 4],
 'fifth': [5],
 'correct': [5, 6]}

# Shunting Yard Algorithm

In [10]:
def setor(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1|myset2
    return myset3

def setand(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1&myset2
    return myset3

def precedence(op): 
      
    if op == '&' or op == '|': 
        return 2
    if op == '!': 
        return 1
    return 0

def applyOp(a,b,op):
    if op=='&':
        return setand(a,b)
    if op=='|':
        return setor(a,b)
    if op=='!':
        return setnot(a,b)
    
def setnot(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1-myset2
    return myset3
    
    
operator=['|', '&','(',')','!']
def evaluate(tokens): 
      
    values = []     
    ops = [] 
    i = 0
      
    while i < len(tokens):           
       
        if tokens[i] == ' ': 
            i += 1
            continue          
      
        elif tokens[i] == '(': 
            ops.append(tokens[i]) 
            
        elif tokens[i] not in operator : 
                      
            values.append(dt[tokens[i]]) 
        elif tokens[i] == ')': 
          
            while len(ops) != 0 and ops[-1] != '(': 
              
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
            
            ops.pop() 
            
            
        else: 
          
           
            while (len(ops) != 0 and
                precedence(ops[-1]) >= precedence(tokens[i])): 
                          
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
           
            ops.append(tokens[i]) 
          
        i += 1
    while len(ops) != 0: 
          
        val2 = values.pop() 
        val1 = values.pop() 
        op = ops.pop() 
                  
        values.append(applyOp(val1, val2, op)) 
      
    
    return values[0] 
          
    

In [12]:
import random
import string
import cherrypy
import nltk
import time
from nltk.tokenize import word_tokenize

from cherrypy import tools


class Root(object):
   
    @cherrypy.expose
    def index(self):
        return """<html>
<head>
  <title>Search</title>
  <style >
    .container{
      margin-left: 260px;
      margin-top: 20%;
      width: 700px;
      height: 50px;
      background-color: lightblue;
      border-top-left-radius: 25px;
      border-bottom-left-radius: 25px;
      border-top-right-radius: 25px;
      border-bottom-right-radius: 25px;
      }
    .container div{
      height: 85%;
      width: 90%;
      margin-left: 5%;
      border-style: hidden;
    }
    .container div input{
      height: 100%;
      width: 100%;
      border-style: hidden;
      margin-top: 5px;
      background-color: transparent;
      font-size: 30px;
      outline: none;
    }
    #submit{
      margin-left: 45%;
      margin-top: 10px;
      height: 40px;
      width: 100px;
    }
  </style>
</head>
<body>
  

  <div class="container">
    <div>
      <form action="store" id="form" onsubmit="return false; method="GET">
         <input id="query" type="text" name="search" placeholder="Enter the query............................................."></div>
  </div><span>
   
    <input id="submit" type="submit" onclick="othername();" /></span>
</form>
<a id="shutdown"; href="./shutdown">Shutdown Server</a>

</body>
</html>"""
    
    
    def store(self, search):
        """Check the username & password"""
        tokens=word_tokenize(search)
        numb=str(evaluate(tokens))
       
        return numb
    
    
    @cherrypy.expose

    def shutdown(self):
        cherrypy.engine.exit()
    
    
    store.exposed = True
    
    @cherrypy.expose

    def shutdown(self):  
        cherrypy.engine.exit()
        

if __name__=="__main__":
    cherrypy.config.update({'server.socket_port': 8089})
    
    cherrypy.quickstart(Root())
   
   
    
  

[18/Jun/2020:02:11:00] ENGINE Listening for SIGTERM.
[18/Jun/2020:02:11:00] ENGINE Bus STARTING
[18/Jun/2020:02:11:00] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[18/Jun/2020:02:11:00] ENGINE Started monitor thread 'Autoreloader'.
[18/Jun/2020:02:11:00] ENGINE Serving on http://127.0.0.1:8089
[18/Jun/2020:02:11:00] ENGINE Bus STARTED


127.0.0.1 - - [18/Jun/2020:02:11:03] "GET / HTTP/1.1" 200 1218 "http://localhost:8888/notebooks/Inverted%20index.ipynb" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
127.0.0.1 - - [18/Jun/2020:02:11:07] "GET /store?search=%28first+%7C+second%29 HTTP/1.1" 200 9 "http://127.0.0.1:8089/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"


[18/Jun/2020:02:11:26] ENGINE Bus STOPPING
[18/Jun/2020:02:11:26] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8089)) shut down
[18/Jun/2020:02:11:26] ENGINE Stopped thread 'Autoreloader'.
[18/Jun/2020:02:11:26] ENGINE Removed handler for console events.
[18/Jun/2020:02:11:26] ENGINE Bus STOPPED
[18/Jun/2020:02:11:26] ENGINE Bus EXITING
[18/Jun/2020:02:11:26] ENGINE Waiting for child threads to terminate...
[18/Jun/2020:02:11:26] ENGINE Bus EXITED


127.0.0.1 - - [18/Jun/2020:02:11:26] "GET /shutdown HTTP/1.1" 200 - "http://127.0.0.1:8089/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"


[18/Jun/2020:02:11:26] ENGINE Waiting for thread CP Server Thread-26.
